# Assignment-6
- Name - Arnab Sen
- En No. - 510519006

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import random
import os
import pandas as pd

from tensorflow.keras import Sequential
from tensorflow.keras.layers import \
    Conv2D, Dense, Input, MaxPool2D, Flatten, \
    Dropout, AvgPool2D, LeakyReLU, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.math import confusion_matrix
import numpy as np
import seaborn as sns

DATASET_DIRECTORY_PATH = '/content/drive/MyDrive/ML_DRIVE/Assign_6/dataset/'
seed = random.randint(0,100)
LRELU_APLHA = 0.01

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" #ignore CUDA messages

### Task-3

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory= DATASET_DIRECTORY_PATH,
    labels="inferred",
    label_mode="categorical", #output of model will be softmax categorical
    color_mode="grayscale",
    batch_size=32,
    image_size=(80,80),
    shuffle=True,
    seed=seed, #same seed for both dataset so that no overlap happens
    validation_split=0.1, #90:10 split
    subset='training',
    crop_to_aspect_ratio=False,
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory= DATASET_DIRECTORY_PATH,
    labels="inferred",
    label_mode="categorical", #output of model will be softmax categorical
    color_mode="grayscale",
    batch_size=32,
    image_size=(80,80),
    shuffle=True,
    seed=seed,
    validation_split=0.1, #90:10 split
    subset='validation',
    crop_to_aspect_ratio=False,
)

## Task 4

In [ ]:
input_shape = (80, 80, 1)
num_class = 5

In [ ]:
def _plot_history(
    history: 'tf.keras.callbacks.History',
    conv_kernels: 'list[tuple[int, int]]',
    conv_filters: 'list[int]',
    activation: 'str',
    pool: 'str',
    num_fc_layers: 'int'
):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.title(f'Loss vs epoch')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training')
    plt.plot(history.history['val_accuracy'], label='Validation')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    plt.title(f'Accuracy vs epoch')

    plt.suptitle(
        f'filters {conv_filters}; kernels {conv_kernels}; {pool}_pool; {activation}; dense layers after flatten= {num_fc_layers}')
    plt.show()


def _plot_confusion_matrix(y_val: 'list[int]',
                           y_pred: 'list[int]'):
    matrix = confusion_matrix(y_val, y_pred)
    fig = plt.figure(figsize=(12, 5))
    sns.heatmap(
        matrix,
        xticklabels=range(1, num_class + 1),
        yticklabels=range(1, num_class + 1),
        linewidth=0.5,
        cmap='coolwarm',
        annot=True,
        cbar=True)
    plt.title('Confusion Matrix for the above model')
    plt.ylabel('Actual Value')
    plt.xlabel('Predicted Value')
    plt.show()


def train_model(
    conv_kernels: 'list[tuple[int, int]]',
    conv_filters: 'list[int]',
    activation: 'str',
    pool: 'str',
    drop_rate: 'float',
    num_fc_layers: 'int',
    train_dataset: "tf.data.Dataset",
    val_dataset: "tf.data.Dataset",
    add_batch_norm=False,
    fc_layer_size=64,
    num_epochs=100,
    extra_conv_layers=0,
    give_model=False
):

    model = Sequential()
    model.add(Input(shape=input_shape))

    for filtr, kernel in zip(conv_filters, conv_kernels):
        if activation == 'lrelu':
            model.add(Conv2D(
                filters=filtr,
                kernel_size=kernel,
                activation=LeakyReLU(alpha=LRELU_APLHA)
            ))
        else:
            model.add(Conv2D(
                filters=filtr,
                kernel_size=kernel,
                activation=activation
            ))

        if pool.lower() == 'max':
            model.add(MaxPool2D())
        elif pool.lower() == 'avg':
            model.add(AvgPool2D())
        else:
            raise Exception('argument pool is undefined')

        if add_batch_norm:
            model.add(BatchNormalization())

        if drop_rate > 0:
            model.add(Dropout(rate=drop_rate))

    for i in range(0, extra_conv_layers):
        conv_filters.append(conv_filters[-1]*2)
        conv_kernels.append(conv_kernels[-1])

        if activation == 'lrelu':
            model.add(Conv2D(
                filters=conv_filters[-1],
                kernel_size=conv_kernels[-1],
                activation=LeakyReLU(alpha=LRELU_APLHA),
                padding='same'
            ))
        else:
            model.add(Conv2D(
                filters=conv_filters[-1],
                kernel_size=conv_kernels[-1],
                activation=activation,
                padding='same'
            ))

    model.add(Flatten())

    for _ in range(num_fc_layers):
        if activation == 'lrelu':
            model.add(Dense(
                units=fc_layer_size,
                activation=LeakyReLU(alpha=LRELU_APLHA)
            ))
        else:
            model.add(Dense(
                units=fc_layer_size,
                activation=activation
            ))

    model.add(Dense(units=num_class, activation='softmax'))


    model.compile(loss=CategoricalCrossentropy(),
                  metrics=['accuracy', Precision(), Recall()])

    callback = [
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ]

    history = model.fit(x=train_dataset,
                        epochs=num_epochs,
                        verbose=0,
                        callbacks=callback if num_epochs == 100 else None,
                        validation_data=val_dataset
                        )

    val_loss, val_accuracy, val_precision, val_recall = \
        model.evaluate(val_dataset, verbose=0)

    _plot_history(
        history=history,
        conv_filters=conv_filters,
        conv_kernels=conv_kernels,
        activation=activation,
        pool=pool,
        num_fc_layers=num_fc_layers
    )

    y_val = [np.argmax(res) for res in np.concatenate(
        [y for x, y in val_dataset], axis=0)]
    y_predict = [np.argmax(res) for res in model.predict(val_dataset)]
    _plot_confusion_matrix(y_val, y_predict)

    if give_model:
        return val_loss, val_accuracy, val_precision, val_recall, model
        
    return val_loss, val_accuracy, val_precision, val_recall


### Task 4.1

In [ ]:
result = pd.DataFrame(columns=[
    'Convolution kernel_size',
    'Convolution filters size',
    'Pooling Layers',
    'Activation',
    'FC layer (after Flatten)',
    'Dropout Rate',
    'Validation Loss',
    'Validation Accuracy',
    'Validation Precision',
    'Validation Recall'
])

convolution_kernels = [
    [(3, 3), (3, 3), (3, 3)],
    [(3, 3), (3, 3), (5, 5)],
    [(3, 3), (5, 5), (5, 5)],
    [(5, 5), (5, 5), (5, 5)],
]
convolution_filters = [16, 32, 64]
activation = 'relu'
dropout_rate = 0.1
num_fc_layers = 0
pool='max'

for kernels in convolution_kernels:
    val_loss, val_acc, val_precision, val_recall = train_model(
        drop_rate=dropout_rate,
        conv_kernels=kernels,
        conv_filters=convolution_filters,
        activation=activation,
        pool=pool,
        num_fc_layers=num_fc_layers,
        train_dataset=train_dataset,
        val_dataset=val_dataset
    )

    result.loc[len(result.index)] = [
        kernels,
        convolution_filters,
        pool,
        activation,
        num_fc_layers,
        dropout_rate,
        val_loss,
        val_acc,
        val_precision, 
        val_recall 
    ]

In [ ]:
result['Validation f1'] = \
    2*result['Validation Precision']*result['Validation Recall']/(result['Validation Precision'] + result['Validation Recall'])
result

In [ ]:
plt.figure(figsize=(12, 5))
plt.bar(
    [str(res) for res in result['Convolution kernel_size']],
    result['Validation Accuracy'])
plt.ylabel('Validation Accuracy')
plt.xlabel('Convolution Kernels')
plt.title("Validation Accuracy for different Convolution Kernel")
plt.show()


In [ ]:
best_conv_kernel = result.sort_values(
    by=['Validation Accuracy', 'Validation Loss'],
    ascending=[False, True]
)['Convolution kernel_size'].iloc[0]

best_conv_kernel


### Task 4.2

In [ ]:
result = pd.DataFrame(columns=[
    'Convolution kernel_size',
    'Convolution filters size',
    'Pooling Layers',
    'Activation',
    'FC layer (after Flatten)',
    'Dropout Rate',
    'Validation Loss',
    'Validation Accuracy',
    'Validation Precision',
    'Validation Recall'
])

convolution_filters = [16, 32, 64]
activation = 'relu'
dropout_rate = 0.1
num_fc_layers = [1, 2, 3]
pool = 'max'

for layers in num_fc_layers:
    val_loss, val_acc, val_precision, val_recall = train_model(
        drop_rate=dropout_rate,
        conv_kernels=best_conv_kernel,
        conv_filters=convolution_filters,
        activation=activation,
        pool=pool,
        num_fc_layers=layers,
        train_dataset=train_dataset,
        val_dataset=val_dataset
    )

    result.loc[len(result.index)] = [
        best_conv_kernel,
        convolution_filters,
        pool,
        activation,
        layers,
        dropout_rate,
        val_loss,
        val_acc,
        val_precision,
        val_recall
    ]


In [ ]:
result['Validation f1'] = \
    2*result['Validation Precision']*result['Validation Recall']/(result['Validation Precision'] + result['Validation Recall'])
result

In [ ]:
plt.figure(figsize=(12, 5))
plt.bar(
    result['FC layer (after Flatten)'],
    result['Validation Accuracy'])
plt.ylabel('Validation Accuracy')
plt.xlabel('FC layer (after Flatten)')
plt.title("Validation Accuracy for different FC layer (after Flatten)")
plt.show()


In [ ]:
best_num_fc_layers = result.sort_values(
    by=['Validation Accuracy', 'Validation Loss'],
    ascending=[False, True]
    )['FC layer (after Flatten)'].iloc[0]

best_num_fc_layers

### Task 4.3

In [ ]:
result = pd.DataFrame(columns=[
    'Convolution kernel_size',
    'Convolution filters size',
    'Pooling Layers',
    'Activation',
    'FC layer (after Flatten)',
    'Dropout Rate',
    'Validation Loss',
    'Validation Accuracy',
    'Validation Precision',
    'Validation Recall'
])

convolution_filters = [16, 32, 64]
activation = 'relu'
dropout_rate = 0.1
pools = ['max', 'avg']

for pool in pools:
    val_loss, val_acc, val_precision, val_recall = train_model(
        drop_rate=dropout_rate,
        conv_kernels=best_conv_kernel,
        conv_filters=convolution_filters,
        activation=activation,
        pool=pool,
        num_fc_layers=best_num_fc_layers,
        train_dataset=train_dataset,
        val_dataset=val_dataset
    )

    result.loc[len(result.index)] = [
        best_conv_kernel,
        convolution_filters,
        pool,
        activation,
        best_num_fc_layers,
        dropout_rate,
        val_loss,
        val_acc,
        val_precision,
        val_recall
    ]


In [ ]:
result['Validation f1'] = \
    2*result['Validation Precision']*result['Validation Recall']/(result['Validation Precision'] + result['Validation Recall'])
result

In [ ]:
plt.figure(figsize=(12, 5))
plt.bar(
    result['Pooling Layers'],
    result['Validation Accuracy'])
plt.ylabel('Validation Accuracy')
plt.xlabel('Pooling Layers')
plt.title("Validation Accuracy for different Pooling Layers")
plt.show()

In [ ]:
best_pool = result.sort_values(
    by=['Validation Accuracy', 'Validation Loss'],
    ascending=[False, True]
    )['Pooling Layers'].iloc[0]

best_pool

### Task 4.4

In [ ]:
result = pd.DataFrame(columns=[
    'Convolution kernel_size',
    'Convolution filters size',
    'Pooling Layers',
    'Activation',
    'FC layer (after Flatten)',
    'Dropout Rate',
    'Validation Loss',
    'Validation Accuracy',
    'Validation Precision',
    'Validation Recall'
])

convolution_filters = [16, 32, 64]
activations = ['sigmoid', 'elu', 'relu', 'lrelu']
dropout_rate = 0.1

for activation in activations:
    val_loss, val_acc, val_precision, val_recall = train_model(
        drop_rate=dropout_rate,
        conv_kernels=best_conv_kernel,
        conv_filters=convolution_filters,
        activation=activation,
        pool=best_pool,
        num_fc_layers=best_num_fc_layers,
        train_dataset=train_dataset,
        val_dataset=val_dataset
    )

    result.loc[len(result.index)] = [
        best_conv_kernel,
        convolution_filters,
        best_pool,
        activation,
        best_num_fc_layers,
        dropout_rate,
        val_loss,
        val_acc,
        val_precision,
        val_recall
    ]


In [ ]:
result['Validation f1'] = \
    2*result['Validation Precision']*result['Validation Recall']/(result['Validation Precision'] + result['Validation Recall'])
result

In [ ]:
plt.figure(figsize=(12, 5))
plt.bar(
    result['Activation'],
    result['Validation Accuracy'])
plt.ylabel('Validation Accuracy')
plt.xlabel('Activation')
plt.title("Validation Accuracy for different Activation")
plt.show()

In [ ]:
best_activation = result.sort_values(
    by=['Validation Accuracy', 'Validation Loss'],
    ascending=[False, True]
    )['Activation'].iloc[0]

best_activation

### Task 4.5

In [ ]:
result = pd.DataFrame(columns=[
    'Convolution kernel_size',
    'Convolution filters size',
    'Pooling Layers',
    'Activation',
    'FC layer (after Flatten)',
    'Dropout Rate',
    'Batch Normalization Present',
    'Validation Loss',
    'Validation Accuracy',
    'Validation Precision',
    'Validation Recall'
])

convolution_filters = [16, 32, 64]
dropout_rates = [0.1, 0.25, 0, 0.1]
batch_norm_cases = [False, False, True, True]

for dropout_rate, do_batch in zip(dropout_rates, batch_norm_cases):
    val_loss, val_acc, val_precision, val_recall = train_model(
        drop_rate=dropout_rate,
        conv_kernels=best_conv_kernel,
        conv_filters=convolution_filters,
        activation=best_activation,
        pool=best_pool,
        add_batch_norm=do_batch,
        num_fc_layers=best_num_fc_layers,
        train_dataset=train_dataset,
        val_dataset=val_dataset
    )

    result.loc[len(result.index)] = [
        best_conv_kernel,
        convolution_filters,
        best_pool,
        best_activation,
        best_num_fc_layers,
        dropout_rate,
        do_batch,
        val_loss,
        val_acc,
        val_precision,
        val_recall
    ]


In [ ]:
result['Validation f1'] = \
    2*result['Validation Precision']*result['Validation Recall']/(result['Validation Precision'] + result['Validation Recall'])
result

In [ ]:
plt.figure(figsize=(12, 5))
plt.bar(
    [f'dropout {drop} batch {batch}' for drop,batch in zip(result['Dropout Rate'], result['Batch Normalization Present'])],
    result['Validation Accuracy'])
plt.ylabel('Validation Accuracy')
plt.xlabel('Regularization')
plt.title("Validation Accuracy for different Regularization")
plt.show()

In [ ]:
best_dropout = result.sort_values(
    by=['Validation Accuracy', 'Validation Loss'],
    ascending=[False, True]
    )['Dropout Rate'].iloc[0]

print(f"best dropout rate: {best_dropout}")

best_do_batch = result.sort_values(
    by=['Validation Accuracy', 'Validation Loss'],
    ascending=[False, True]
    )['Batch Normalization Present'].iloc[0]

print(f"Batch Normalization present for best case: {best_do_batch}")

### Task 4.6

In [ ]:
import time

result = pd.DataFrame(columns=[
    'Convolution kernel_size',
    'Convolution filters size',
    'Pooling Layers',
    'Activation',
    'FC layer (after Flatten)',
    'Dropout Rate',
    'Batch Normalization Present',
    'Extra Conv Layers',
    'Validation Loss',
    'Validation Accuracy',
    'Validation Precision',
    'Validation Recall',
    'Time Taken'
])

convolution_filters = [16, 32, 64]
extra_conv_layers = [0, 1, 2, 3]

for extra_conv_layer in extra_conv_layers:
    # making a deep copy to avoid editing of original element
    convolution_filters_copy = []
    best_conv_kernel_copy = []

    for filtr, kernel in zip(convolution_filters, best_conv_kernel):
        convolution_filters_copy.append(filtr)
        best_conv_kernel_copy.append(kernel)

    start_time = time.time()

    val_loss, val_acc, val_precision, val_recall = train_model(
        drop_rate=best_dropout,
        conv_kernels=best_conv_kernel_copy,
        conv_filters=convolution_filters_copy,
        activation=best_activation,
        pool=best_pool,
        add_batch_norm=best_do_batch,
        num_fc_layers=best_num_fc_layers,
        extra_conv_layers=extra_conv_layer,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
    )
    end_time = time.time()

    time_taken = end_time - start_time

    result.loc[len(result.index)] = [
        best_conv_kernel_copy,
        convolution_filters_copy,
        best_pool,
        best_activation,
        best_num_fc_layers,
        best_dropout,
        best_do_batch,
        extra_conv_layer,
        val_loss,
        val_acc,
        val_precision,
        val_recall,
        time_taken
    ]


In [ ]:
result['Validation f1'] = \
    2*result['Validation Precision']*result['Validation Recall']/(result['Validation Precision'] + result['Validation Recall'])
result

In [ ]:
plt.figure(figsize=(12, 5))
plt.bar(
    result['Extra Conv Layers'],
    result['Validation Accuracy'])
plt.ylabel('Validation Accuracy')
plt.xlabel('Extra Conv Layers')
plt.title("Validation Accuracy for different Extra Conv Layers")
plt.show()

In [ ]:
best_extra_conv_layer = result.sort_values(
    by=['Validation Accuracy', 'Validation Loss'],
    ascending=[False, True]
    )['Extra Conv Layers'].iloc[0]

best_extra_conv_layer

### Task 4.7

For the best set of parameters obtained here repeat the experimentation for color images. And visualize the test result.

In [ ]:
result = pd.DataFrame(columns=[
    'Convolution kernel_size',
    'Convolution filters size',
    'Pooling Layers',
    'Activation',
    'FC layer (after Flatten)',
    'Dropout Rate',
    'Batch Normalization Present',
    'Extra Conv Layers',
    'Color Mode',
    'Validation Loss',
    'Validation Accuracy',
    'Validation Precision',
    'Validation Recall',
])

convolution_filters = [16, 32, 64]

val_loss, val_acc, val_precision, val_recall = train_model(
    drop_rate=best_dropout,
    conv_kernels=best_conv_kernel,
    conv_filters=convolution_filters,
    activation=best_activation,
    pool=best_pool,
    add_batch_norm=best_do_batch,
    num_fc_layers=best_num_fc_layers,
    extra_conv_layers=best_extra_conv_layer,
    train_dataset=train_dataset,
    val_dataset=val_dataset
)

result.loc[len(result.index)] = [
    best_conv_kernel,
    convolution_filters,
    best_pool,
    best_activation,
    best_num_fc_layers,
    best_dropout,
    best_do_batch,
    best_extra_conv_layer,
    'grayscale',
    val_loss,
    val_acc,
    val_precision,
    val_recall

]

convolution_filters = [16, 32, 64]

color_train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=DATASET_DIRECTORY_PATH,
    labels="inferred",
    label_mode="categorical",  # output of model will be softmax categorical
    color_mode="rgb",
    batch_size=32,
    image_size=(80, 80),
    shuffle=True,
    seed=seed,  # same seed for both dataset so that no overlap happens
    validation_split=0.1,  # 90:10 split
    subset='training',
    crop_to_aspect_ratio=False,
)

color_val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=DATASET_DIRECTORY_PATH,
    labels="inferred",
    label_mode="categorical",  # output of model will be softmax categorical
    color_mode="rgb",
    batch_size=32,
    image_size=(80, 80),
    shuffle=True,
    seed=seed,
    validation_split=0.1,  # 90:10 split
    subset='validation',
    crop_to_aspect_ratio=False,
)

val_loss, val_acc, val_precision, val_recall = train_model(
    drop_rate=best_dropout,
    conv_kernels=best_conv_kernel,
    conv_filters=convolution_filters,
    activation=best_activation,
    pool=best_pool,
    add_batch_norm=best_do_batch,
    num_fc_layers=best_num_fc_layers,
    extra_conv_layers=best_extra_conv_layer,
    train_dataset=train_dataset,
    val_dataset=val_dataset
)

result.loc[len(result.index)] = [
    best_conv_kernel,
    convolution_filters,
    best_pool,
    best_activation,
    best_num_fc_layers,
    best_dropout,
    best_do_batch,
    best_extra_conv_layer,
    'rgb',
    val_loss,
    val_acc,
    val_precision,
    val_recall
]


In [ ]:
result['Validation f1'] = \
    2*result['Validation Precision']*result['Validation Recall']/(result['Validation Precision'] + result['Validation Recall'])
result

In [ ]:
plt.figure(figsize=(12, 5))
plt.bar(
    result['Color Mode'],
    result['Validation Accuracy'])
plt.ylabel('Validation Accuracy')
plt.xlabel('Color Mode')
plt.title("Validation Accuracy for different Color Mode")
plt.show()

## Task 6

In [ ]:
import numpy as np
import tensorflow as tf

def create_dataset(X, Y, batch_size=32):
  """ Create train and test TF dataset from X and Y
    The prefetch overlays the preprocessing and model execution of a training step. 
    While the model is executing training step s, the input pipeline is reading the data for step s+1.
    AUTOTUNE automatically tune the number for sample which are prefeteched automatically. 
    
    Keyword arguments:
    X -- numpy array
    Y -- numpy array
    batch_size -- integer
  """
  AUTOTUNE = tf.data.experimental.AUTOTUNE
  
  dataset = tf.data.Dataset.from_tensor_slices((X, Y))
  dataset = dataset.shuffle(buffer_size=1000, reshuffle_each_iteration=True)
  dataset = dataset.batch(batch_size).prefetch(AUTOTUNE)
  
  return dataset

In [ ]:
import pandas as pd
from tensorflow.image import resize

mnist_data = tf.keras.datasets.mnist.load_data()
(x_train, y_train), (x_test, y_test) = mnist_data

#taking less data due to memory limitation
x_train = x_train[0:10000]
x_test = x_test[0:2000]
y_train = y_train[0:10000]
y_test = y_test[0:2000]

#resizing to 80x80 and mapping 0-255 to 0-1
x_train = np.reshape(x_train, (-1, 28, 28, 1))
x_train = x_train/255
x_train = resize(x_train, (80,80))


x_test = np.reshape(x_test, (-1, 28, 28, 1))
x_test = x_test/255
x_test = resize(x_test, (80,80))

y_train = pd.get_dummies(y_train).to_numpy()
y_test = pd.get_dummies(y_test).to_numpy()

mnist_train_dataset=create_dataset(x_train, y_train)
mnist_val_dataset=create_dataset(x_test, y_test)


In [ ]:
convolution_filters = [16, 32, 64]
num_class = 10

val_loss, val_acc, val_precision, val_recall, model = train_model(
    drop_rate=best_dropout,
    conv_kernels=best_conv_kernel,
    conv_filters=convolution_filters,
    activation=best_activation,
    pool=best_pool,
    add_batch_norm=best_do_batch,
    num_fc_layers=best_num_fc_layers,
    extra_conv_layers=best_extra_conv_layer,
    train_dataset=mnist_train_dataset,
    val_dataset=mnist_val_dataset,
    give_model=True
)

print(f"val_loss= {val_loss}\nval_acc={val_acc}\nval_precision={val_precision}\nval_recall={val_recall}")

In [ ]:
import random

random_idx = random.sample(range(0, len(x_test)), 10)

img_to_predict = np.array([x_test[idx] for idx in random_idx])

categorical_predictions = model.predict(img_to_predict)

for img, cat_pred in zip(img_to_predict, categorical_predictions):
    plt.imshow(img, cmap='gray')
    plt.show()
    pred = np.argmax(cat_pred)
    print(f"predict = {pred}")